### Download Grammar

In [1]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [2]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [3]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Node Types

In [4]:
print(bert_tokenizer.node_types)

['ellipsis', 'pattern', 'exec', 'pass', 'call', 'expression_statement', 'assignment', 'as', 'dictionary_comprehension', 'block', 'expression', 'format_expression', 'while_statement', '/', 'argument_list', ':', 'case_pattern', 'from', '&', '!=', 'print_statement', 'type_conversion', '_simple_statement', 'async', '-', 'keyword_argument', 'keyword_separator', 'positional_separator', 'break_statement', 'match_statement', 'unary_operator', 'except', '<<', '**=', '<', 'import', 'exec_statement', '^=', '@', 'except_clause', 'dictionary_splat_pattern', 'continue', 'concatenated_string', 'is', 'parameter', 'import_prefix', 'list_splat_pattern', 'pair', 'class', 'yield', 'return', '-=', 'format_specifier', '=', 'set', 'binary_operator', 'list_splat', 'else', '_compound_statement', 'case', 'typed_default_parameter', 'delete_statement', 'lambda_parameters', '>>=', 'print', 'relative_import', 'subscript', 'primary_expression', '{', 'boolean_operator', 'case_clause', 'string', 'escape_sequence', 'im

### Encodings

In [5]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
#code = "def hello_world(a,b):\n    print('hello world')"
#code = "def __ordered_values_by_indexes(self, data, inds): \"\"\" Return values (intensities) by indexes. Used for multiscale graph cut. data = [[0 1 1], [0 2 2], [0 2 2]] inds = [[0 1 2], [3 4 4], [5 4 4]] return: [0, 1, 1, 0, 2, 0] If the data are not consistent, it will take the maximal value \"\"\" # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
#code = "def __ordered_values_by_indexes(self, data, inds):  # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
target_node_type = "*"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type))

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def multiply_numbers(a,b):
    return a<mask>b


### Code Prediction

In [6]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

### Evaluation

In [7]:
import CodeCheckList.utils as utils

prediction_number = 0
print('------------- CODE -------------')
print(code)
print('\n---------- MASKED -------------')
print(masked_code)
print('\n--------- PREDICTED -----------')
predicted_code = predictions[prediction_number]
print(predicted_code)
print('\n--------- AST COMPARE -----------')
filtered_nodes = []
filtered_nodes_predict = []
utils.find_nodes(bert_tokenizer.parser.parse(bytes(code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes)
utils.find_nodes(bert_tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes_predict)
print(len(filtered_nodes))
print(len(filtered_nodes_predict))
#base the evaluation on size comparison

------------- CODE -------------
def multiply_numbers(a,b):
    return a*b

---------- MASKED -------------
def multiply_numbers(a,b):
    return a<mask>b

--------- PREDICTED -----------
def multiply_numbers(a,b):
    return a,b

--------- AST COMPARE -----------
1
0


In [12]:
from CodeCheckList.evaluator import Evaluator

evaluator = Evaluator(checkpoint, python_language)

results = evaluator(code, 5)

print(results)

[{'ellipsis': []}, {'pattern': []}, {'exec': []}, {'pass': []}, {'call': []}, {'expression_statement': []}, {'assignment': []}, {'as': []}, {'dictionary_comprehension': []}, {'block': [(1, 1, True), (1, 1, True), (1, 1, True), (1, 1, True), (1, 1, True)]}, {'expression': []}, {'format_expression': []}, {'while_statement': []}, {'/': []}, {'argument_list': []}, {':': [(1, 1, True), (1, 1, True), (1, 1, True), (1, 1, True), (1, 1, True)]}, {'case_pattern': []}, {'from': []}, {'&': []}, {'!=': []}, {'print_statement': []}, {'type_conversion': []}, {'_simple_statement': []}, {'async': []}, {'-': []}, {'keyword_argument': []}, {'keyword_separator': []}, {'positional_separator': []}, {'break_statement': []}, {'match_statement': []}, {'unary_operator': []}, {'except': []}, {'<<': []}, {'**=': []}, {'<': []}, {'import': []}, {'exec_statement': []}, {'^=': []}, {'@': []}, {'except_clause': []}, {'dictionary_splat_pattern': []}, {'continue': []}, {'concatenated_string': []}, {'is': []}, {'parame

### Module

In [9]:
#| default_exp evaluator

In [10]:
#| export
import CodeCheckList

import CodeCheckList.utils as utils
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker
from CodeCheckList.predictor import Predictor

In [11]:
#| export
class Evaluator:
    """Evaluator Module to perform all AST Evaluations"""
    def __init__(self, checkpoint: str, language):
        self.tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)
        self.masker = Masker(self.tokenizer)
        self.predictor = Predictor.from_pretrained(checkpoint, self.tokenizer)

    def __call__(self, source_code: str, number_of_predictions: int):
        evaluation_results = []
        for node_type_idx, node_type in enumerate(self.tokenizer.node_types):
            evaluation_results.append({node_type: self.evaluate_snippet(source_code, node_type_idx, number_of_predictions)})
        return evaluation_results
            
    def evaluate_snippet(self, source_code: str, target_node_type_idx: int, number_of_predictions: int):
        results=[]
        source_code_nodes = []
        utils.find_nodes(self.tokenizer.parser.parse(bytes(source_code, "utf8")).root_node, 
            self.tokenizer.node_types[target_node_type_idx], source_code_nodes)
        if len(source_code_nodes) == 0:
            return results

        #source_code_encoding = self.tokenizer(source_code)
        masked_code_encoding = self.masker(source_code, self.tokenizer(source_code), target_node_type_idx)
        #masked_code = self.tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == self.tokenizer.tokenizer.bos_token_id or 
        #    token_id == self.tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))
        predictions = self.predictor(masked_code_encoding, self.tokenizer.tokenizer(source_code, return_tensors='pt'), number_of_predictions)       

        for prediction_number in range(0, number_of_predictions):
            predicted_code = predictions[prediction_number]
            predicted_nodes = []
            utils.find_nodes(self.tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, self.tokenizer.node_types[target_node_type_idx], predicted_nodes)
            results.append((len(source_code_nodes), len(predicted_nodes), len(predicted_nodes)>=len(source_code_nodes)))

        return results